In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset,DataLoader

import os
import cv2
import numpy as np
import pandas as pd

from PIL import Image
import plotly.express as px
from tqdm.notebook import tqdm

import gdown
from utils.imputer import Imputer
from utils.fashion_dataset import FashionDataset
from models.inception import Model

**Downloading The Deep Fashion Dataset**

In [2]:
#gdown.download(id="1p7-dU6rDuqZ2mxv5ac5AWndt4z19aS6j")
#!unzip "classification-assignment.zip"
#!rm -r "__MACOSX/"
#!rm -r "classification-assignment.zip"

In [3]:
df = pd.read_csv("./classification-assignment/attributes.csv")
mask = pd.Series([os.path.exists(os.path.join('./classification-assignment/images/',df.loc[id,"filename"])) for id in df.index])
df = df[mask].reset_index(drop=True)
df.head()

,filename,neck,sleeve_length,pattern
0,cdc8cd2a-0938-4970-a3b5-f5ed9595222c1527925869...,6.0,NaN,4.0
1,11469770662809-Metersbonwe-Navy-T-shirt-485146...,5.0,3.0,9.0
2,f7ad67ab-eeb1-4449-8f63-7b580d2797e71532342804...,NaN,0.0,9.0
3,11516770810185-Splash-Men-Tshirts-767151677081...,6.0,3.0,9.0
4,11505295751483-FOREVER-21-Men-White-Self-Desig...,1.0,3.0,NaN


**Fill The Missing Values using Imputer**

In [4]:
imputer = Imputer()
df=imputer.impute(df=df)
df=df.dropna().reset_index(drop=True)
df.head()

,filename,neck,sleeve_length,pattern
0,cdc8cd2a-0938-4970-a3b5-f5ed9595222c1527925869...,6.0,3.0,4.0
1,11469770662809-Metersbonwe-Navy-T-shirt-485146...,5.0,3.0,9.0
2,f7ad67ab-eeb1-4449-8f63-7b580d2797e71532342804...,6.0,0.0,9.0
3,11516770810185-Splash-Men-Tshirts-767151677081...,6.0,3.0,9.0
4,11505295751483-FOREVER-21-Men-White-Self-Desig...,1.0,3.0,9.0


**Building & Splitting Dataset from Utils**

In [5]:
ds = FashionDataset(df=df)
split = int(0.9*len(ds))
ds_train,ds_val = torch.utils.data.random_split(dataset=ds,lengths=[split,len(ds)-split])

**Loading Model & Other Hyperparameters**

In [6]:
device="cuda"
model = Model().to(device)

In [7]:
# Weights Calculations...
vals = df.neck.value_counts().sum()-df.neck.value_counts()
neck_weights =torch.tensor(((vals)/np.sum(vals)).values,dtype=torch.float32).to(device)

vals = df.sleeve_length.value_counts().sum()-df.sleeve_length.value_counts()
sleeve_weights =torch.tensor(((vals)/np.sum(vals)).values,dtype=torch.float32).to(device)

vals = df.pattern.value_counts().sum()-df.pattern.value_counts()
pattern_weights =torch.tensor(((vals)/np.sum(vals)).values,dtype=torch.float32).to(device)

loss_fn1 = nn.CrossEntropyLoss(weight=neck_weights).to(device)
loss_fn2 = nn.CrossEntropyLoss(weight =sleeve_weights).to(device)
loss_fn3 = nn.CrossEntropyLoss(weight=pattern_weights).to(device)

batch_size=32

In [8]:
optim = torch.optim.Adam(params=model.parameters(),lr=0.0001) 

In [9]:
def get_accuracy(ypred,ytrue):
    count = 0
    for i in range(ytrue.shape[0]):
        if torch.argmax(ypred[i,:]) == ytrue[i] : 
            count+=1 
    return count/ytrue.shape[0]

**Training The Model**

In [ ]:
for epoch in range(0,100):
    ds_iter = iter(DataLoader(dataset=ds_train,batch_size=batch_size,shuffle=True))
    pbar = tqdm(range(len(ds_iter)))
    losses = []
    accs = []
    for batch in pbar : 
        x,y = next(ds_iter)
        x = x.to(device)
        y = y.to(device)
        y1,y2,y3 = y[:,0],y[:,1],y[:,2]

        ypred1,ypred2,ypred3 = model(x)

        loss_neck = loss_fn1(ypred1,y1)
        loss_sleeve = loss_fn2(ypred2,y2)
        loss_pattern = loss_fn3(ypred3,y3)

        acc_neck = get_accuracy(ypred1,y1)
        acc_sleeve = get_accuracy(ypred2,y2)
        acc_pattern = get_accuracy(ypred3,y3)

        acc = np.mean([acc_neck,acc_sleeve,acc_pattern])

        loss = loss_neck+loss_sleeve+loss_pattern

        optim.zero_grad()
        loss.backward()
        optim.step()

        losses.append(loss.data.cpu().numpy())
        accs.append(acc)
        
        pbar.set_postfix({"Loss ":np.mean(losses),"Acc":np.mean(accs)})
        
    # Validation Step ...
    model.eval()
    ds_iter = iter(DataLoader(dataset=ds_val,batch_size=batch_size,shuffle=True))
    val_losses = []
    val_accs = []
    for batch in range(len(ds_iter)): 
        x,y = next(ds_iter)
        x = x.to(device)
        y = y.to(device)
        y1,y2,y3 = y[:,0],y[:,1],y[:,2]

        ypred1,ypred2,ypred3 = model(x,eval_=True)

        loss_neck = loss_fn1(ypred1,y1)
        loss_sleeve = loss_fn2(ypred2,y2)
        loss_pattern = loss_fn3(ypred3,y3)

        acc_neck = get_accuracy(ypred1,y1)
        acc_sleeve = get_accuracy(ypred2,y2)
        acc_pattern = get_accuracy(ypred3,y3)

        acc = np.mean([acc_neck,acc_sleeve,acc_pattern])

        loss = loss_neck+loss_sleeve+loss_pattern

        optim.zero_grad()
        loss.backward()
        optim.step()

        val_losses.append(loss.data.cpu().numpy())
        val_accs.append(acc)

    model.train()
    print({"Val Loss": np.mean(val_losses),"Val Acc":np.mean(val_accs)})

  0%|          | 0/37 [00:00<?, ?it/s]

{'Val Loss': 2.8993964, 'Val Acc': 0.70625}


  0%|          | 0/37 [00:00<?, ?it/s]

{'Val Loss': 2.5188406, 'Val Acc': 0.7041666666666666}


  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

{'Val Loss': 0.03375333, 'Val Acc': 0.9958333333333332}


  0%|          | 0/37 [00:00<?, ?it/s]

{'Val Loss': 0.03891271, 'Val Acc': 0.99375}


  0%|          | 0/37 [00:00<?, ?it/s]